In [ ]:
!pip install git+https://github.com/RQC-Robotics/RQC-Robotics-tactile_sensor

In [ ]:
import sensor_lib as sl
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
os.mkdir('generated_gauses')
os.mkdir('models_weights')
os.mkdir('resalts')

In [ ]:
n_gauses=15
n_pic=100000
n_fibers=4
n_rot=1
lerning_rate=1e-4
n_epochs=20

dir_of_gauses='/content/drive/MyDrive/Colab_projects/'

# generation pressure_map

In [ ]:
sl.generate_pressure_map(
    n_pic, n_fibers, x= 97, y = 97,
    part='generated_gauses/fresh_gauss'+str(n_gauses)+'_'+str(n_pic)+'.npy')

# counting losses in fibers (input of nerual network)

In [ ]:
input, output, input_test, output_test = sl.sim_on_gpu(
    dir_of_gauses+'fresh_gauss'+str(n_gauses)+'_'+str(n_pic)+'.npy',
    n_random_rot=n_rot, n_angles=n_fibers,
    batch_size_preproc=128*8
    )
input_shape = input.shape
output_shape = output.shape
dataset_b, dataset = sl.prepare_dataset_for_train(
    input, output, batch_size_fit_model=1024*2
    )
dataset_test_b, dataset_test = sl.prepare_dataset_for_train(
    input_test, output_test,batch_size_fit_model=1024*2
    )

# defining nerual network for decoding and fitting it

In [ ]:
model = sl.SensorNN4S(input_shape[1:3], output_shape[1:3])
model.build(input_shape)
model.summary()
model.compile(
    optimizer=tf.keras.optimizers.Adam(lerning_rate),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['accuracy']
    )

In [ ]:
model.fit(dataset_b, epochs = n_epochs, verbose=1)

In [ ]:
model.save('models_weights/model4S_'+str(n_fibers)+'fi.nn')

# evoluate model on don't seen data

In [ ]:
model = tf.keras.models.load_model('models_weights/model4S_'+str(n_fibers)+'fi.nn')

In [ ]:
model.evaluate(dataset_test_b)

In [ ]:
predictions = model.predict(dataset_test_b)
predictions.shape

# saving resalts

In [ ]:
with open('resalts/pred_4S_'+str(n_gauses)+'g_'+str(n_fibers)+'fi', 'wb') as f:
    np.save(f, predictions)
    np.save(f, output_test)

# look on resalts

In [ ]:
with open('resalts/pred_4S_'+str(n_gauses)+'g_'+str(n_fibers)+'fi', 'rb') as f:
    predictions = np.load(f)
    output_test = np.load(f)

In [ ]:
N = 4 #28 26 24 23 21 20
plt.imshow(predictions[N])
plt.show()
plt.imshow(output_test[N])
plt.show()